In [1]:
from pathlib import Path, PurePath
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode
from analysis import read_behaviors, read_moea_fitness, unpickle_moea
from utility.util_functions import calc_str_disparity, euclidean_distance
from utility.visualize import plot_path, plot_moea_fitness, plot_surrogate_model, plot_str_disparity
import pickle
import numpy as np
from deap import base, creator
from evolution.simulation import Disparity
import scipy as sp
from scipy import stats
import math
init_notebook_mode(connected=True)


ModuleNotFoundError: DBus works only on linux!


Using TensorFlow backend.


ModuleNotFoundError: DBus works only on linux!


In [ ]:
# get the path for all behavioral features
files = sorted(Path('data/results/').glob('transferability_*/behavioral_features.dat'))
fitness_f = sorted(Path('data/results/').glob('transferability_*/fitness.dat'))

In [ ]:
columns = [
        'avg_left', 'avg_right',
        's1', 's2', 's3', 's4', 's5', 's6', 's7',
        'area0_percentage',
        'area1_percentage',
        'area2_percentage',
    ]

In [ ]:
fitness_f[8]

In [ ]:
behaviors = read_behaviors([files[8]])[0]

In [ ]:
behaviors.loc[behaviors['genome_id'] == 358].head()

In [ ]:
behaviors.head()

In [ ]:
b_grouped = behaviors.groupby(['genome_id']).filter(lambda x: len(x) > 1).groupby(['genome_id', 'simulation']).first()

In [ ]:
b_grouped.head()

In [ ]:
euclidean_distance(b_grouped[columns].iloc[1], b_grouped[columns].iloc[0])

In [ ]:
for i in range(1, len(b_grouped), 2):
    print('genome_id:', 
          b_grouped[columns].iloc[i].name[0],
          'actual diversity',
          euclidean_distance(
              b_grouped[columns].iloc[i],
              b_grouped[columns].iloc[i-1]
          )
    )

In [ ]:
fitness_data = read_moea_fitness([fitness_f[8]])[0]

In [ ]:
fitness_data.sort_values(by=['fitness'])

In [ ]:
fitness_data.head()

In [ ]:
hof = unpickle_moea('data/results/transferability_simulation_17/hof')

In [ ]:
len(hof)

In [ ]:
plot_moea_fitness(fitness_data, hof)

In [ ]:
hof_ids = []
for ind in hof:
    hof_ids.append(ind.key)
    print(ind.key, ind.fitness.values)

In [2]:
genomes_file = sorted(Path('data/results/transferability_simulation_17/deap_inds').glob('*_genome.pkl'))
transfered_file = sorted(Path('data/results/transferability_simulation_17/deap_inds').glob('*_transformed_genome_.pkl'))
genomes = [unpickle_moea(genome) for genome in genomes_file]
transferred = [unpickle_moea(genome) for genome in transfered_file]

/Users/richban/.local/share/virtualenvs/behavioral.neuroevolution-ViNSkuNA/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning:

A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.

/Users/richban/.local/share/virtualenvs/behavioral.neuroevolution-ViNSkuNA/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning:

A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.



In [ ]:
transfered_file

In [ ]:
len(transfered_file)

In [3]:
g = sorted(genomes, key=lambda x: x.key)

In [4]:
t = sorted(transferred, key=lambda x: x.key)

In [5]:
sim_genome = list(filter(lambda x: x.key == 448, g))[0]

In [6]:
print(sim_genome.key, sim_genome.fitness.values, sim_genome.sim_time)

448 (5.44717889947933, 1.5418591521249227, 0.3075231201931484) 20.400000000000155


In [7]:
trans_genome = list(filter(lambda x: x.key == 448, t))[0]

In [8]:
print(trans_genome.key, trans_genome.fitness.values)

448 (5.44717889947933, 1.5418591521249227, 0.3075231201931484)


In [ ]:
sim_genome.sim_time

In [ ]:
sim_genomes = sorted(list(filter(lambda x: x.key in hof_ids[:-10], g)), key=lambda x: x.fitness.values[0])
trans_genomes = sorted(list(filter(lambda x: x.key in hof_ids[:-10] t)), key=lambda x: x.fitness.values[0])

In [9]:
plot_path([sim_genome, trans_genome])
# plot_path(sim_genomes)

/Users/richban/.local/share/virtualenvs/behavioral.neuroevolution-ViNSkuNA/lib/python3.7/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [10]:
sim_genome.position

array([[0.20800586, 0.22291239],
       [0.26569957, 0.22599578],
       [0.32288846, 0.22944501],
       [0.38050514, 0.23386016],
       [0.43908486, 0.2389268 ],
       [0.49716806, 0.24529591],
       [0.55573905, 0.2517474 ],
       [0.61369956, 0.2583614 ],
       [0.67167222, 0.26662892],
       [0.72957069, 0.27524209],
       [0.78783381, 0.28352749]])

In [11]:
trans_genome.position

[[0.19, 0.22],
 array([0.27922436, 0.20181334]),
 array([0.38534854, 0.21712335]),
 array([0.49635221, 0.24065779]),
 array([0.61948073, 0.27334344]),
 array([0.72863136, 0.3096033 ]),
 array([0.83616897, 0.35416614]),
 array([0.93792116, 0.40601272]),
 array([1.03371195, 0.46460683]),
 array([1.08276178, 0.51529906]),
 array([1.08367588, 0.53012542])]

In [ ]:
plot_path(trans_genomes)

In [12]:
calc_str_disparity(trans_genome.position, sim_genome.position)

4.144616178771767

In [ ]:
euclidean_distance(trans_genome.position, sim_genome.position)

In [ ]:
plot_surrogate_model(fitness_data)

In [ ]:
# merge  
union_t = [(x, y) for x in g for y in t if x.key == y.key]
for s, t in union_t:
    np.array_equal(s, t)
    assert s.key == t.key

In [ ]:
str_disparity = Disparity()

In [ ]:
for s, t in union_t:
    print(s.gen, end=" ")

In [ ]:
str_disparities = [(s.gen, s.key, s.fitness.values[0], s.fitness.values[1], calc_str_disparity(t.position, s.position)) for s, t in union_t]    

In [ ]:
sorted(str_disparities, key=lambda x: x[0])

In [ ]:
np.array([str_disparity[1] for str_disparity in str_disparities])
np.array([real_disparity[2] for real_disparity in str_disparities])

filter_disparities = list(filter(lambda x: x[2] < 5, str_disparities))
filter_disparities[:4]

In [ ]:
plot_str_disparity(str_disparities)

In [ ]:
actual = []
approximated = []
for s, t in union_t:
    approximated.append(s.fitness.values[1])
    actual.append(calc_str_disparity(t.position, s.position))

In [ ]:
stats.pearsonr(actual, approximated)

In [ ]:
distance = 0
for i in range(0, len(trans_genome.position)-1):
    distance += math.sqrt(((sim_genome.position[i][0] - 
                            sim_genome.position[i][1])**2) + 
                          ((sim_genome.position[i+1][1] - 
                            sim_genome.position[i+1][1])**2))

In [ ]:
distance